In [35]:
from datetime import datetime
import pandas as pd
from geopy import distance
import numpy as np

import pymongo
from pymongo import MongoClient

import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

In [36]:
client = MongoClient('localhost', 27017)

db = client['avl_pipeline_test']
in_coll = db['trips']
out_coll = db['edges']

In [37]:
# Optional - Clean the output collection!
out_coll.delete_many({});

In [38]:
sample = in_coll.find_one()
edge_list = list(sample['edges'].keys())

In [39]:
for edge in edge_list:
    
    edge_dict = {}
    
    edge_dict['sequence'] = edge
    
    search_str = 'edges.' + edge
    prj_str = search_str + '.edge_time'

    search = {
        search_str: { '$exists': True }
    }
    
    projection = {
        '_id': 0, 
        prj_str: 1
        }

    edge_duration_list = []

    for edge_duration in in_coll.find(search, projection):

        edge_time = edge_duration['edges'][edge]['edge_time']
        edge_duration_list.append(edge_time)
        
    np_edge = np.array(edge_duration_list)
    
    edge_dict['mean'] = np_edge.mean()
    edge_dict['std_dev'] = np.std(np_edge)
    
    out_coll.insert_one(edge_dict)
    

In [40]:
out_coll.find_one()

{'_id': ObjectId('5ae7b4ba3ad39e762bcbecfb'),
 'sequence': '1',
 'mean': 81.58061953931691,
 'std_dev': 38.841898301128815}